In [30]:
import openai

In [31]:
client = openai.OpenAI(api_key = "여기에 API 키를 넣어주세요")

In [32]:
vector_store = client.beta.vector_stores.create(name="축구 규칙 파일")

In [33]:
file_streams = open("축구규칙정리.pdf", "rb")

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=[file_streams]
)
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [34]:
instruction = '''
[목적]
이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.

[규칙]
1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.
2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.
3. 답변의 형태는 아래 예시와 같이 해주세요
예시)
- 질문 : 질문 내용
- 답변 : 답변 내용
4. 모든 질문에 한국어로 답변해주세요.
'''
assistant = client.beta.assistants.create(
    name = "축구 규칙 설명 챗봇",
    instructions=instruction,
    model="gpt-3.5-turbo-1106",
    tools=[{"type" : "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

print(assistant.id)

asst_TAVMrWBHr6j5jU95OdJBnM7G


In [35]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_TAVMrWBHr6j5jU95OdJBnM7G', created_at=1726722149, description=None, instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n예시)\n- 질문 : 질문 내용\n- 답변 : 답변 내용\n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-3.5-turbo-1106', name='축구 규칙 설명 챗봇', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_w8o8VJZpKY0QRsk7Go23Pf5t'])), top_p=1.0), Assistant(id='asst_wXxbbHGtQUNS9honhW1WR4m3', created_at=1726715950, description=None, instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드

In [36]:
assistant = client.beta.assistants.retrieve("assistant ID")

In [37]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_1WJ24V3EmMSwPwfLNzXUBpKN', created_at=1726722166, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [38]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="축구장의 크기는?"
)
print(message)

Message(id='msg_T9Vn3yuUEAcphovieUL5dkXJ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='축구장의 크기는?'), type='text')], created_at=1726722174, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_1WJ24V3EmMSwPwfLNzXUBpKN')


In [39]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [40]:
import time

while run.status not in ["completed", "failed"]:
    run = client.beta.threads.runs.retrieve(
        thread_id= thread.id,
        run_id= run.id
    )
    print(run.status)
    time.sleep(5)

in_progress
completed


In [41]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [42]:
for each in messages:
    print(each.role + ": "+each.content[0].text.value)
    print("=========")

assistant: 축구장의 크기는 다음과 같습니다.

- 터치라인의 길이: 최소 90m (100 야드) ~ 최대 120m (130 야드)
- 골라인의 길이: 최소 45m (50 야드) ~ 최대 90m (100 야드)
- 국제경기를 위한 터치라인의 길이: 최소 100m (110 야드) ~ 최대 110m (120 야드)
- 국제경기를 위한 골라인의 길이: 최소 64m (70 야드) ~ 최대 75m (80 야드) 

이 정보는 "축구규칙정리.pdf" 파일에서 찾아낸 것입니다.    
user: 축구장의 크기는?


In [43]:
# 메시지 추가
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="오프사이드 룰에 대해 설명해줘"
)

# 메시지 리스트 가져오기
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# 메시지 프린트
for each in messages:
    print(each.role + ": "+each.content[0].text.value)
    print("=========")

user: 오프사이드 룰에 대해 설명해줘
assistant: 축구장의 크기는 다음과 같습니다.

- 터치라인의 길이: 최소 90m (100 야드) ~ 최대 120m (130 야드)
- 골라인의 길이: 최소 45m (50 야드) ~ 최대 90m (100 야드)
- 국제경기를 위한 터치라인의 길이: 최소 100m (110 야드) ~ 최대 110m (120 야드)
- 국제경기를 위한 골라인의 길이: 최소 64m (70 야드) ~ 최대 75m (80 야드) 

이 정보는 "축구규칙정리.pdf" 파일에서 찾아낸 것입니다.    
user: 축구장의 크기는?


In [44]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

while run.status not in ["completed", "failed"]:
    
    run = client.beta.threads.runs.retrieve(
        thread_id= thread.id,
        run_id= run.id
    )

    print(run.status)
    
    time.sleep(5)

in_progress
completed


In [45]:
# 메시지 리스트 가져오기
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# 메시지 프린트
for each in messages:
    print(each.role + ": "+each.content[0].text.value)
    print("=========")

assistant: 오프사이드 규칙에 대해 설명해드리겠습니다. 축구에서 오프사이드는 상대방 골지역에 더 많은 선수를 두는 것을 막기 위한 규칙입니다. 공격 선수가 볼을 받을 때 골 지역에 더 많은 수비수가 있거나, 공격 선수가 볼을 받을 때 이미 골 지역 안에 있을 때 더 많은 수비수를 가로막는 경우에 오프사이드가 선언됩니다. 따라서, 선수는 상대방의 수비수보다 공격선수가 더 떨어져 있어야 합니다. 오프사이드를 위반한 경우 해당 팀은 프리킥을 얻게 됩니다. 선수들의 위치에 따라 다양한 상황에서 오프사이드가 적용될 수 있습니다.

이 정보는 "축구규칙정리.pdf" 파일에서 찾은 내용입니다.【0:오프사이드 규칙】
user: 오프사이드 룰에 대해 설명해줘
assistant: 축구장의 크기는 다음과 같습니다.

- 터치라인의 길이: 최소 90m (100 야드) ~ 최대 120m (130 야드)
- 골라인의 길이: 최소 45m (50 야드) ~ 최대 90m (100 야드)
- 국제경기를 위한 터치라인의 길이: 최소 100m (110 야드) ~ 최대 110m (120 야드)
- 국제경기를 위한 골라인의 길이: 최소 64m (70 야드) ~ 최대 75m (80 야드) 

이 정보는 "축구규칙정리.pdf" 파일에서 찾아낸 것입니다.    
user: 축구장의 크기는?


In [46]:
# assistant 삭제
response = client.beta.assistants.delete(assistant.id)
print(response)

AssistantDeleted(id='asst_TAVMrWBHr6j5jU95OdJBnM7G', deleted=True, object='assistant.deleted')


In [47]:
# 업로드한 파일 목록 가져오기
file_list = client.files.list()
print(file_list)

SyncPage[FileObject](data=[FileObject(id='file-gzVClBMqxYQNDDm9nN1a10dJ', bytes=22073717, created_at=1726722142, filename='축구규칙정리.pdf', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-AUsCAwXwXGQ9VQzciCq96nJP', bytes=22073717, created_at=1726715000, filename='축구규칙정리.pdf', object='file', purpose='assistants', status='processed', status_details=None)], object='list', has_more=False)


In [48]:
# 특정 파일 ID 가져오기
file_id = file_list.data[0].id
print(file_id)

file-gzVClBMqxYQNDDm9nN1a10dJ


In [49]:
# 업로드 파일 삭제
response = client.files.delete(file_id)
print(response)

FileDeleted(id='file-gzVClBMqxYQNDDm9nN1a10dJ', deleted=True, object='file')
